# Oxygen Sensor Dataset — Exploratory Data Analysis (EDA)

This notebook performs exploratory data analysis on the oxygen saturation dataset provided in `oxygen.csv`.

The goals are to:
- Understand the structure and contents of the dataset.
- Explore time coverage and sampling frequency.
- Analyse missing values and data quality.
- Inspect the distribution of `Oxygen[%sat]` globally and per sensor.
- Understand the sensor layout (Systems, Units, EquipmentUnits, SubUnits).
- Visualise time-series patterns and identify potential gaps or irregularities.

We will treat each unique combination of `(System, EquipmentUnit, SubUnit)` as a logical **sensor time series**.


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pathlib import Path

# Plotting style
plt.style.use("seaborn-v0_8-whitegrid")

# Display settings for pandas
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

In [ ]:
DATA_DIR = os.path.abspath(os.path.join(os.getcwd(), "../" , "data"))

DATA_RAW_DIR = Path(os.path.join(DATA_DIR, "raw"))

In [ ]:
# Path to the dataset (adjust this if needed)
DATA_PATH = Path(os.path.join(DATA_RAW_DIR, "oxygen.csv"))

# Read the CSV
df_raw = pd.read_csv(DATA_PATH)

print("Raw shape:", df_raw.shape)
df_raw.head()

In [ ]:
df = df_raw.copy()

# Drop an unnecessary index-like column if present
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# Parse time column to datetime
df["time"] = pd.to_datetime(df["time"], errors="coerce")

# Basic sort by time
df = df.sort_values("time").reset_index(drop=True)

print("Cleaned shape:", df.shape)
df.head()

In [ ]:
print("=== DataFrame Info ===")
df.info()

print("\n=== Basic stats for numeric columns ===")
display(df.describe(include=[np.number]))

print("\n=== Basic stats for categorical columns ===")
display(df.describe(include=["object"]))

In [ ]:
id_cols = ["System", "Unit", "EquipmentUnit", "SubUnit"]
for col in id_cols:
    if col in df.columns:
        print(f"\nColumn: {col}")
        print(f"  Unique values: {df[col].nunique()}")
        print("  Top 10 value_counts:")
        display(df[col].value_counts().head(10))

In [ ]:
min_time = df["time"].min()
max_time = df["time"].max()

print("Time coverage")
print("  Earliest timestamp:", min_time)
print("  Latest timestamp:", max_time)
print("  Total duration:", max_time - min_time)

print("\nRow count:", len(df))

# Quick look at number of rows per calendar date
df["date"] = df["time"].dt.date
rows_per_date = df["date"].value_counts().sort_index()
print("\nRows per date:")
display(rows_per_date)

In [ ]:
# Fill SubUnit to avoid NaN in concatenation
df["SubUnit_filled"] = df["SubUnit"].fillna("NA").astype(str)

df["sensor_id"] = (
    df["System"].astype(str)
    + "|" + df["EquipmentUnit"].astype(str)
    + "|" + df["SubUnit_filled"]
)

print("Number of distinct sensors:", df["sensor_id"].nunique())

sensor_counts = df["sensor_id"].value_counts()
print("\nSensor_id value counts (top 10):")
display(sensor_counts.head(10))

print("\nSummary of rows per sensor:")
display(sensor_counts.describe())

In [ ]:
# Sort by sensor and time
df = df.sort_values(["sensor_id", "time"]).reset_index(drop=True)

# Compute time difference in seconds per sensor
df["time_diff_sec"] = (
    df.groupby("sensor_id")["time"]
      .diff()
      .dt.total_seconds()
)

print("Global distribution of time_diff_sec (ignoring NaNs):")
display(df["time_diff_sec"].describe())

print("\nTop value counts for time_diff_sec:")
display(df["time_diff_sec"].value_counts().head(10))

# Summarise sampling intervals per sensor
sampling_summary = (
    df.groupby("sensor_id")["time_diff_sec"]
      .agg(
          count_non_null="count",
          median_diff_sec="median",
          min_diff_sec="min",
          max_diff_sec="max"
      )
      .sort_values("median_diff_sec")
)

print("\nSampling interval summary per sensor (first 10 sensors):")
display(sampling_summary.head(10))

In [ ]:
print("=== Missing values (absolute & %) by column ===")
missing_abs = df.isna().sum()
missing_pct = df.isna().mean() * 100

missing_df = pd.DataFrame({
    "missing_count": missing_abs,
    "missing_pct": missing_pct
}).sort_values("missing_pct", ascending=False)

display(missing_df)

In [ ]:
if "Oxygen[%sat]" in df.columns:
    missing_sensor = (
        df.groupby("sensor_id")["Oxygen[%sat]"]
          .agg(
              total_rows="size",
              missing_count=lambda s: s.isna().sum(),
              missing_pct=lambda s: s.isna().mean() * 100
          )
          .sort_values("missing_pct", ascending=True)
    )

    print("Missingness of Oxygen[%sat] per sensor (lowest missing first):")
    display(missing_sensor.head(10))

    print("\nSensors with the highest missing %:")
    display(missing_sensor.tail(10))

In [ ]:
if "Oxygen[%sat]" in df.columns:
    # per day
    daily_missing = (
        df.groupby("date")["Oxygen[%sat]"]
          .apply(lambda s: s.isna().mean() * 100)
          .rename("missing_pct")
    )

    print("Daily % of missing Oxygen[%sat]:")
    display(daily_missing)

    plt.figure(figsize=(10, 4))
    daily_missing.plot(marker="o")
    plt.title("Daily Missing % of Oxygen[%sat]")
    plt.ylabel("Missing %")
    plt.xlabel("Date")
    plt.tight_layout()
    plt.show()

    # per hour
    df["hour"] = df["time"].dt.hour
    hourly_missing = (
        df.groupby(["date", "hour"])["Oxygen[%sat]"]
          .apply(lambda s: s.isna().mean() * 100)
          .reset_index(name="missing_pct")
    )

    print("\nHourly missing % sample (first 20 rows):")
    display(hourly_missing.head(20))

In [ ]:
if "Oxygen[%sat]" in df.columns:
    print("Global stats for Oxygen[%sat]:")
    display(df["Oxygen[%sat]"].describe())

    plt.figure(figsize=(8, 4))
    df["Oxygen[%sat]"].hist(bins=30)
    plt.title("Histogram of Oxygen[%sat] (all sensors)")
    plt.xlabel("Oxygen[%sat]")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(4, 6))
    df.boxplot(column="Oxygen[%sat]")
    plt.title("Boxplot of Oxygen[%sat] (all sensors)")
    plt.tight_layout()
    plt.show()

In [ ]:
if "Oxygen[%sat]" in df.columns:
    # Aggregate per System
    system_stats = (
        df.groupby("System")["Oxygen[%sat]"]
          .describe()
          .sort_values("mean")
    )
    print("Oxygen[%sat] stats by System:")
    display(system_stats)

    # Aggregate per Unit
    unit_stats = (
        df.groupby("Unit")["Oxygen[%sat]"]
          .describe()
          .sort_values("mean")
    )
    print("\nOxygen[%sat] stats by Unit:")
    display(unit_stats)

    # Boxplot per System
    plt.figure(figsize=(10, 4))
    df_non_null = df.dropna(subset=["Oxygen[%sat]"])
    df_non_null.boxplot(column="Oxygen[%sat]", by="System")
    plt.title("Oxygen[%sat] by System")
    plt.suptitle("")
    plt.xlabel("System")
    plt.ylabel("Oxygen[%sat]")
    plt.tight_layout()
    plt.show()

In [ ]:
if "Oxygen[%sat]" in df.columns:
    ts_df = df.set_index("time")

    # Compute mean oxygen across all sensors at each timestamp (ignoring NaNs)
    oxygen_mean_ts = ts_df["Oxygen[%sat]"].resample("5T").mean()  # 5-minute resample

    plt.figure(figsize=(12, 4))
    oxygen_mean_ts.plot()
    plt.title("Mean Oxygen[%sat] over Time (5-minute resample)")
    plt.xlabel("Time")
    plt.ylabel("Mean Oxygen[%sat]")
    plt.tight_layout()
    plt.show()

In [ ]:
if "Oxygen[%sat]" in df.columns:
    # Choose a few sensors with the least missing Oxygen
    example_sensors = (
        df.groupby("sensor_id")["Oxygen[%sat]"]
          .apply(lambda s: s.isna().mean())
          .sort_values()
          .head(3)
          .index.tolist()
    )

    print("Example sensors with relatively complete data:")
    for s in example_sensors:
        print(" ", s)

    ts_df = df.set_index("time")

    plt.figure(figsize=(12, 6))
    for sensor in example_sensors:
        sensor_series = ts_df[ts_df["sensor_id"] == sensor]["Oxygen[%sat]"]
        plt.plot(sensor_series.index, sensor_series.values, label=sensor)
    plt.title("Oxygen[%sat] over Time for Selected Sensors")
    plt.xlabel("Time")
    plt.ylabel("Oxygen[%sat]")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
if "Oxygen[%sat]" in df.columns:
    # Define plausible operating range (adjust as needed)
    plausible_min = 0
    plausible_max = 200

    out_of_range_mask = (
        (df["Oxygen[%sat]"] < plausible_min)
        | (df["Oxygen[%sat]"] > plausible_max)
    )

    num_out_of_range = out_of_range_mask.sum()
    print(f"Number of oxygen readings outside [{plausible_min}, {plausible_max}]: {num_out_of_range}")

    if num_out_of_range > 0:
        print("\nSample of out-of-range readings:")
        display(df.loc[out_of_range_mask].head(20))

In [ ]:
# For each sensor, summarise time gaps > expected interval (e.g. > 60 seconds)
expected_interval_sec = 60

gap_df = (
    df[df["time_diff_sec"].notna()]
    .assign(is_large_gap=lambda x: x["time_diff_sec"] > expected_interval_sec)
)

gap_summary = (
    gap_df.groupby("sensor_id")["is_large_gap"]
      .agg(
          num_large_gaps="sum",
          total_transitions="count"
      )
)

gap_summary["large_gap_pct"] = (
    gap_summary["num_large_gaps"] / gap_summary["total_transitions"] * 100
)

print("Large gap summary per sensor (sorted by % of large gaps):")
display(gap_summary.sort_values("large_gap_pct", ascending=False))

## Quick EDA Summary (to be filled in after running)

Use this section to write down key observations after inspecting the plots and tables above:
- Time coverage and sampling frequency.
- Sensors with complete vs highly missing data.
- Typical range and distribution of Oxygen[%sat].
- Presence of gaps, flatlines, or irregularities.


In [ ]:
summary_notes = """
- Add your bullet-point notes here after reviewing the EDA output.
"""
print(summary_notes)